In [7]:
# Importing the necessary libraries for data manipulation and reading
import pandas as pd
# Importing necessary libraries for logistic regression and scaling
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import log_loss
# Load the data
training_data = pd.read_csv('training_data.csv')

# Drop rows where BORROWER_ID is 'xNullx'
training_data = training_data[training_data['BORROWER_ID'] != 'xNullx']

# Shuffle the DataFrame
training_data = training_data.sample(frac=1, random_state=42).reset_index(drop=True)

data_submission_example = pd.read_csv('data_submission_example.csv')


In [8]:
lognormal_variables = [
    'CONTRACT_CREDIT_LOSS', 'CONTRACT_DEPT_SERVICE_TO_INCOME',
    'CONTRACT_INCOME', 'CONTRACT_INSTALMENT_AMOUNT', 'CONTRACT_INSTALMENT_AMOUNT_2',
    'CONTRACT_LOAN_AMOUNT', 'CONTRACT_MARKET_VALUE', 'CONTRACT_MORTGAGE_LENDING_VALUE',
]
date_variables = ['CONTRACT_DATE_OF_LOAN_AGREEMENT', 'CONTRACT_MATURITY_DATE']

In [9]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming training_data =  training_data.sample(n=10000, random_state=1)

# Filling NaN values with 0
training_data.fillna(0, inplace=True)

# Converting columns to numeric where possible
for col in training_data.columns:
    try:
        training_data[col] = pd.to_numeric(training_data[col], errors='ignore')
    except:
        continue

# Creating a sample target variable
training_data['TARGET_EVENT_BINARY'] = np.where(training_data['TARGET_EVENT'] == 'K', 1, 0)

In [10]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming training_data =  training_data.sample(n=10000, random_state=1)

# Filling NaN values with 0
training_data.fillna(0, inplace=True)

# Converting columns to numeric where possible
for col in training_data.columns:
    try:
        training_data[col] = pd.to_numeric(training_data[col], errors='ignore')
    except:
        continue

# Creating a sample target variable
training_data['TARGET_EVENT_BINARY'] = np.where(training_data['TARGET_EVENT'] == 'K', 1, 0)

In [11]:
training_data['TARGET_EVENT_DAY_DATETIME'] = pd.to_datetime(training_data['TARGET_EVENT_DAY'])
training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'] = pd.to_datetime(training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT'])

# Calculate the day difference
training_data['DAY_DIFF'] = (training_data['TARGET_EVENT_DAY_DATETIME'] - training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME']).dt.days

training_data['TARGET_EVENT_BINARY_1Y'] = np.where(
        (training_data['TARGET_EVENT'] == 'K') & 
        (training_data['DAY_DIFF'] <= 365) & 
        (training_data['DAY_DIFF'] >= 0), 
        1, 
        0
    )
# Drop the temporary 'DAY_DIFF' column if needed
training_data.drop('DAY_DIFF', axis=1, inplace=True)

In [12]:
for var in date_variables:
    training_data[var+ '_JULIAN'] = pd.to_datetime(training_data[var], origin='julian', unit='D')

training_data['TIME_TO_MATURITY'] = (training_data['CONTRACT_MATURITY_DATE_JULIAN'] - training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT_JULIAN']).dt.days

year_2018_date = pd.Timestamp('2018-01-01')
training_data['TIME_TO_MATURITY'] = (training_data['CONTRACT_MATURITY_DATE_JULIAN'] - year_2018_date).dt.days
training_data['ADJUSTED_TIME_TO_MATURITY'] = training_data['TIME_TO_MATURITY'].apply(lambda x: max(min(730, x),0))

In [13]:
start_date = pd.Timestamp('2015-01-01')
end_date = pd.Timestamp('2017-01-01')

# Mask for rows with CONTRACT_DATE_OF_LOAN_AGREEMENT_JULIAN between start_date and end_date
mask_date_range = (training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT_JULIAN'] >= start_date) & (training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT_JULIAN'] <= end_date)

# Mask for rows with TARGET_EVENT_DAY after end_date
mask_future_event = (training_data['TARGET_EVENT_DAY'] > 2017) | (training_data['TARGET_EVENT_DAY'] == 0)

# Combine the masks
combined_mask = mask_date_range & mask_future_event

# Create the new DataFrame with filtered rows
training_data_test = training_data.loc[combined_mask]
training_data_train = training_data.loc[mask_date_range]

In [14]:
grouped_data = training_data_test.groupby('BORROWER_ID')['TARGET_EVENT_BINARY_1Y'].apply(max).reset_index()

C:\Users\takat\AppData\Local\Temp\ipykernel_10908\3860391630.py:1: FutureWarning: The provided callable <built-in function max> is currently using np.maximum.reduce. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass np.maximum.reduce instead.
  grouped_data = training_data_test.groupby('BORROWER_ID')['TARGET_EVENT_BINARY_1Y'].apply(max).reset_index()


In [15]:
grouped_data.columns = ['BORROWER_ID', 'TARGET_EVENT_BINARY_1Y']
grouped_data.to_csv('data/1y-test.csv', index=False)

In [17]:
grouped_data.value_counts('BORROWER_ID').value_counts()

count
1    808147
Name: count, dtype: int64